# FUNCTION GENERATOR using Policy Gradient

Useful links:
Policy Gradient Explanation: http://karpathy.github.io/2016/05/31/rl/ <br>
Example of Policy Gradient: https://github.com/keon/policy-gradient

In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import TimeDistributed, Dense, Reshape, Flatten, GRU, Input, Embedding
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from PolicyGradientModel import PolicyGradientModel
from RewardCalculator import RewardCalculator

Using TensorFlow backend.


In [2]:
ALLOWED_PARAMETERS = list('XY')
ALLOWED_SYMBOLS = ALLOWED_PARAMETERS + list('0123456789+-*/#')
NUM_SYMBOLS = len(ALLOWED_SYMBOLS)
MAX_LENGTH = 30 # Max length of the output expression
CORRECT_EXPRESSION = "3*X+2*Y"

### DEFINE MODEL

In [3]:
def getModel():
    # Trying to neglect input
    input1 = Input(shape=(1,1))
    # TODO: Add noise layer to make output vary
    x = GRU(32)(input1)
    out = Dense(NUM_SYMBOLS, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                loss='categorical_crossentropy')
    return model

In [4]:
rewardCalculator = RewardCalculator(correctExpression=CORRECT_EXPRESSION,
                                    parameters=ALLOWED_PARAMETERS,
                                    usingFunctionDifferenceReward=False,
                                    usingCompilableReward=True,
                                    usingLengthReward=False,
                                    usingFile=False)

In [5]:
model = getModel()
pgModel = PolicyGradientModel(model=model,
                              allowedSymbol=ALLOWED_SYMBOLS,
                              numSymbol=NUM_SYMBOLS,
                              maxLength=MAX_LENGTH,
                              rewardCalculator=rewardCalculator,
                              learningRate=0.0001,
                              fileName="Model1.hdf5")

## TRAINING

In [6]:
pgModel.train(input=np.ones((1,1,1)))

Epoch: 0	Loss: 24.4627891541	Example Output: -
Saving Weight
Epoch: 1	Loss: 24.4581098557	Example Output: 35/114805*09*95581*-/*/X
Epoch: 2	Loss: 24.451915741	Example Output: /*88/0Y34298-532+27/Y
Epoch: 3	Loss: 24.4740461349	Example Output: -*26-23/+/4Y559642-9/-51/37*
Epoch: 4	Loss: 24.5024398804	Example Output: 12849YY82Y85*++050*9
Epoch: 5	Loss: 24.5196674347	Example Output: 4*2/13X06+133-Y4-69Y+18//X
Epoch: 6	Loss: 24.5335128784	Example Output: 132-269-3-6+7XX9
Epoch: 7	Loss: 24.5467790604	Example Output: 15X
Epoch: 8	Loss: 24.5501754761	Example Output: 8090X686082Y57Y5Y2
Epoch: 9	Loss: 24.5505758286	Example Output: X44+628*308X5*X971*
Epoch: 10	Loss: 24.5450210571	Example Output: 25+/9-62**9
Saving Weight
Epoch: 11	Loss: 24.5446870804	Example Output: 
Epoch: 12	Loss: 24.5474981308	Example Output: /5X
Epoch: 13	Loss: 24.5546567917	Example Output: 
Epoch: 14	Loss: 24.5742706299	Example Output: 1Y
Epoch: 15	Loss: 24.5912818909	Example Output: ++970/07-4*+2044140/6359++9
Epoch: 16	Lo

Epoch: 120	Loss: 25.4788125992	Example Output: 9/2255YYY*5340105Y17/82121793*
Saving Weight
Epoch: 121	Loss: 25.4809288025	Example Output: 094+880-683541+86X6-+X30*498*4
Epoch: 122	Loss: 25.4742307663	Example Output: 0
Epoch: 123	Loss: 25.4398500443	Example Output: **226+63+*78-11/33008087662581
Epoch: 124	Loss: 25.3984495163	Example Output: 35385123+++4Y5-X+107+*Y1-Y3712
Epoch: 125	Loss: 25.3713077545	Example Output: X01+-Y49-576*24678927Y9-30-091
Epoch: 126	Loss: 25.3552513123	Example Output: 237/+4780870Y9236Y2/95806245+
Epoch: 127	Loss: 25.3759803772	Example Output: Y85-*/26+60Y6+983Y641X11682287
Epoch: 128	Loss: 25.4100456238	Example Output: 40
Epoch: 129	Loss: 25.4086921692	Example Output: 5429771+31Y*X2381087+746337+5
Epoch: 130	Loss: 25.3742446899	Example Output: 5*9-57/8222X9
Saving Weight
Epoch: 131	Loss: 25.3388507843	Example Output: 885+98*3-*198Y3676+944301/0611
Epoch: 132	Loss: 25.3143215179	Example Output: +/*-8*25136290882X-78-4-915*48
Epoch: 133	Loss: 25.2715467453	Exa

Epoch: 230	Loss: 23.089622879	Example Output: 1838798*8188089X801170509793-9
Saving Weight
Epoch: 231	Loss: 23.0335334778	Example Output: 31431815116-165657550977825571
Epoch: 232	Loss: 22.9947637558	Example Output: 2146-5+5+84877+9498152632934-+
Epoch: 233	Loss: 22.9630716324	Example Output: 2679854255/382-6628
Epoch: 234	Loss: 22.9263935089	Example Output: 3572752894324934+6918778-12016
Epoch: 235	Loss: 22.8686368942	Example Output: 26-263382851378413/23986832600
Epoch: 236	Loss: 22.7877859116	Example Output: 08876338+6613-4193+847-6486961
Epoch: 237	Loss: 22.7235006332	Example Output: 438118011541+5383657++5217-47-
Epoch: 238	Loss: 22.6821941376	Example Output: 641+5+257218468322-+529236/948
Epoch: 239	Loss: 22.6541854858	Example Output: 5727815837268562553-88749+4447
Epoch: 240	Loss: 22.6219013214	Example Output: 89977512922881555+738513801+13
Saving Weight
Epoch: 241	Loss: 22.5620588303	Example Output: -2-878343243888+538-6389789252
Epoch: 242	Loss: 22.5075727463	Example Output: 7

Epoch: 340	Loss: 20.0922485352	Example Output: 341222593882111361699636298925
Saving Weight
Epoch: 341	Loss: 20.1341407776	Example Output: 459662566392834739598535334252
Epoch: 342	Loss: 20.1655729294	Example Output: 12372382242125363119798+574957
Epoch: 343	Loss: 20.1774505615	Example Output: 938111445611177938514865138488
Epoch: 344	Loss: 20.1822963715	Example Output: 32522879476866+812858127678899
Epoch: 345	Loss: 20.1663990021	Example Output: 9413934379217743
Epoch: 346	Loss: 20.1424474716	Example Output: 57228147*426-98398293253
Epoch: 347	Loss: 20.118719101	Example Output: 88
Epoch: 348	Loss: 20.1150966644	Example Output: 473
Epoch: 349	Loss: 20.0986125946	Example Output: 12-8
Epoch: 350	Loss: 20.0884057999	Example Output: 5551663241713819885*3983856396
Saving Weight
Epoch: 351	Loss: 20.0959787369	Example Output: 273
Epoch: 352	Loss: 20.08885746	Example Output: 1
Epoch: 353	Loss: 20.0758096695	Example Output: 3561623552+16234646+422748
Epoch: 354	Loss: 20.0666713715	Example Outpu

Epoch: 469	Loss: 19.1743228912	Example Output: 728988998616441222-47372184984
Epoch: 470	Loss: 19.2128887177	Example Output: 8296848847769399327
Saving Weight
Epoch: 471	Loss: 19.2629831314	Example Output: +
Epoch: 472	Loss: 19.2535022736	Example Output: 46755734
Epoch: 473	Loss: 19.2222923279	Example Output: 1143196
Epoch: 474	Loss: 19.1919435501	Example Output: 1
Epoch: 475	Loss: 19.1529649734	Example Output: 24392649746711447954915
Epoch: 476	Loss: 19.1354997635	Example Output: 8
Epoch: 477	Loss: 19.0943777084	Example Output: 77996747
Epoch: 478	Loss: 19.0353193283	Example Output: 
Epoch: 479	Loss: 18.9986602783	Example Output: 
Epoch: 480	Loss: 18.9805511475	Example Output: 35965326886
Saving Weight
Epoch: 481	Loss: 18.9706930161	Example Output: 276
Epoch: 482	Loss: 18.9462352753	Example Output: 648972583
Epoch: 483	Loss: 18.9251773834	Example Output: 537113868248145719544415329599
Epoch: 484	Loss: 18.9029172897	Example Output: 34166326879411892
Epoch: 485	Loss: 18.8785974503	Examp

...

Epoch: 5694	Loss: 19.3181852341	Example Output: 
Epoch: 5695	Loss: 19.292805481	Example Output: 699424
Epoch: 5696	Loss: 19.2554803848	Example Output: 1
Epoch: 5697	Loss: 19.2328714371	Example Output: 958195
Epoch: 5698	Loss: 19.2486183167	Example Output: 7247312277532163941
Epoch: 5699	Loss: 19.2752754211	Example Output: 464
Epoch: 5700	Loss: 19.2715484619	Example Output: 336491917633943
Saving Weight
Epoch: 5701	Loss: 19.2686889648	Example Output: 1168
Epoch: 5702	Loss: 19.2675848007	Example Output: 759379742782745721195371175173
Epoch: 5703	Loss: 19.2491470337	Example Output: 
Epoch: 5704	Loss: 19.2118095398	Example Output: 127211466943677626136955752653
Epoch: 5705	Loss: 19.152807045	Example Output: 379793233663643414149215526866
Epoch: 5706	Loss: 19.1073959351	Example Output: 61953562113
Epoch: 5707	Loss: 19.0716464996	Example Output: 6438634375347271164818
Epoch: 5708	Loss: 19.0426727295	Example Output: 8235334533999419
Epoch: 5709	Loss: 19.0325292587	Example Output: 6879
Epoch: 

Epoch: 5827	Loss: 18.6132123947	Example Output: 
Epoch: 5828	Loss: 18.5833559036	Example Output: 44715619
Epoch: 5829	Loss: 18.5812400818	Example Output: 294172396529
Epoch: 5830	Loss: 18.6125959396	Example Output: 152235623136353
Saving Weight
Epoch: 5831	Loss: 18.6594150543	Example Output: 5531571776629679
Epoch: 5832	Loss: 18.6738397598	Example Output: 926121651
Epoch: 5833	Loss: 18.6605148315	Example Output: 845465
Epoch: 5834	Loss: 18.6388118744	Example Output: 35934
Epoch: 5835	Loss: 18.6265417099	Example Output: 552171496
Epoch: 5836	Loss: 18.6257743835	Example Output: 935771622416434
Epoch: 5837	Loss: 18.6079589844	Example Output: 299137394715245626
Epoch: 5838	Loss: 18.6165489197	Example Output: 765447125223771753
Epoch: 5839	Loss: 18.6599977493	Example Output: 352754256561763959437761236495
Epoch: 5840	Loss: 18.6543876648	Example Output: 52917315697224294
Saving Weight
Epoch: 5841	Loss: 18.6485292435	Example Output: 
Epoch: 5842	Loss: 18.6588352203	Example Output: 7756
Epoch:

Epoch: 5961	Loss: 18.5059530258	Example Output: 911
Epoch: 5962	Loss: 18.4758377075	Example Output: 2772
Epoch: 5963	Loss: 18.4672039032	Example Output: 6
Epoch: 5964	Loss: 18.4661882401	Example Output: 279982351294761354947515
Epoch: 5965	Loss: 18.458001709	Example Output: 153184193715193669361674117779
Epoch: 5966	Loss: 18.463948822	Example Output: 71629251
Epoch: 5967	Loss: 18.4499902725	Example Output: 69731
Epoch: 5968	Loss: 18.4626041412	Example Output: 9953743995966
Epoch: 5969	Loss: 18.4680339813	Example Output: 1574132337753777567124337
Epoch: 5970	Loss: 18.4603345871	Example Output: 5
Saving Weight
Epoch: 5971	Loss: 18.4569076538	Example Output: 196475419636555799
Epoch: 5972	Loss: 18.452995491	Example Output: 2371576742416172452
Epoch: 5973	Loss: 18.4458400726	Example Output: 236
Epoch: 5974	Loss: 18.5073242188	Example Output: 915477
Epoch: 5975	Loss: 18.5378499985	Example Output: 1469551229916575673
Epoch: 5976	Loss: 18.5153556824	Example Output: 3972915
Epoch: 5977	Loss: 1

Epoch: 6092	Loss: 18.6410587311	Example Output: 177
Epoch: 6093	Loss: 18.6288709641	Example Output: 65926172
Epoch: 6094	Loss: 18.6149114609	Example Output: 1
Epoch: 6095	Loss: 18.6114124298	Example Output: 
Epoch: 6096	Loss: 18.6032150269	Example Output: 6
Epoch: 6097	Loss: 18.5938741684	Example Output: 
Epoch: 6098	Loss: 18.5713096619	Example Output: 5449231895
Epoch: 6099	Loss: 18.5792770386	Example Output: 264149276653912
Epoch: 6100	Loss: 18.5989023209	Example Output: 3414475919
Saving Weight
Epoch: 6101	Loss: 18.6067386627	Example Output: 4459
Epoch: 6102	Loss: 18.6089729309	Example Output: 6691697
Epoch: 6103	Loss: 18.6097703934	Example Output: 192916911
Epoch: 6104	Loss: 18.5975975037	Example Output: 17793269615815425771
Epoch: 6105	Loss: 18.5754264832	Example Output: 9755
Epoch: 6106	Loss: 18.5642471313	Example Output: 115794399712
Epoch: 6107	Loss: 18.5757335663	Example Output: 799
Epoch: 6108	Loss: 18.5696199417	Example Output: 292
Epoch: 6109	Loss: 18.5323915482	Example Out

KeyboardInterrupt: 